# Binary_Classification_Preprocessing
    
1. load
2. resize
3. rotate
4. store
    
Load the image as `mode L` for 8-bit gray scale
    
Resize the image to specified values using antialias filter
    
Load the images into an array.
    
Rotate original image seven times 45$^{\circ}$ and load each one into the df. This will give a seven fold increase in the training images. This will dependedent on the specific directory

Each time an image is loaded into the dataframe, record the ground truth for that image in a target array


This notebook was used to create the various datasources for the binary classification. 

Various image sizes and amounts were used.

In [ ]:
# All the necessary imports, no bloat here!
import os
import time
import numpy as np
from PIL import Image

In [ ]:
# process_images loads, converts, rotates, flattens, and stores the images

def process_images(filename, img_array, target, directory):
    
# Open the image and convert to 8bit grayscale
    image = Image.open(filename).convert('L')
    
# Resize image with antialias filter
    image = image.resize((width, height), Image.ANTIALIAS)
    
# Flatten the matrix to an array and then reshape to a row vector
    flatimage = np.ravel(np.array(image)).reshape((1,length))
    
# Stack starter image onto the array and update target
    img_array = np.vstack((img_array, flatimage))
    target.append(directory)
    
    
# Create a 45degree tilt 7 times for more images, flatten and stack, update target
    for i in range(1,8):
        newimage = image.rotate((45 * i))
        flatnewimage = np.ravel(np.array(newimage)).reshape((1,length))
        img_array = np.vstack((img_array, flatnewimage))
        target.append(directory)
        newimage.close()
        image.close()

        if (len(img_array) % 1000 == 0):
            print(f'{len(img_array)} images processed')
            
    return img_array

In [ ]:
# From the given directory, step though each file. If it's an image, process it

def get_images(directory, img_array, target):

# Create the complete image directory
    imgdir = '../../images/train/' + directory + '/'

    i = 0;
    
# Step through the files, processing the .jpg's
    for file in os.listdir(imgdir):

        filename = os.fsdecode(file)
        if filename.endswith(".jpg"):
            fullfile = imgdir + filename
            img_array = process_images(fullfile, img_array, target, directory)
        else:
            continue
        
        
# Limiter, currenty twice # of mc's
        if i == 4000:
            break
         
        i += 1
                                 
    return img_array

## Get the images and store them in the dataframe.

Step through the image folders in order

In [ ]:
# Constants

# List of directory names
directory_list = ['car', 'motorcycle']

# height and width of images, length of flattened array
width = height = 28
length = width * height

num_dir = len(directory_list)

# save file name. If you needed a comment to figure this one out, go lie down and put a wet towel on your head
img_save_file = ('../../data/imagefile')
target_save_file = ('../../data/targetfile')

In [ ]:
# Get the first image and use it to create the array and target list
# I don't know why, but I can't figure out how to create an empty array, so I'll create
# an array with the first image

# open and convert to 8bit grayscale
imagefile = '../../images/train/car/00000004.jpg'
im1 = Image.open(imagefile).convert('L')


# resize with antialias filter
im1 = im1.resize((width, height), Image.ANTIALIAS)


# reshape
image_array = np.ravel(np.array(im1)).reshape((1,length))


# Close the image and release it
im1.close()


# Create the targetlist
image_target = ['car']

In [ ]:
# Step throught the directory list and get the images in each directory
start_time = time.time()
print(f'Getting {(1982 + 4000) * 8} images.')

for imgdir in directory_list:
    
    image_array = get_images(imgdir, image_array, image_target)
    
elapsed_time = time.time() - start_time
print(f'Shape of image_array: {image_array.shape}')
print(f'Length of target: {len(image_target)}')
print(f'Size of image_array in MB {sys.getsizeof(image_array) / 1024**2}')
print(f'Total Time: {np.round(elapsed_time,2)}s')

# Write the array out
np.save(img_save_file, image_array)
print(f'File saved to: {img_save_file}')

# Write target array
np.save(target_save_file, image_target)
print(f'Target saved to: {target_save_file}')